# StyleTTS2 設定ファイル生成

RTX 4070ti (VRAM 12GB) 環境に最適化された設定ファイルを各データセットごとに生成します。

## 最適化パラメータ

- **batch_size**: 6 (VRAM 12GBに最適化)
- **max_len**: 300 (メモリ使用量削減)
- **epochs_1st**: 80 (Stage 1)
- **epochs_2nd**: 40 (Stage 2)
- **fp16_run**: true (Mixed precision)
- **save_freq**: 10

## 出力

各データセットごとに以下を生成:
- `StyleTTS2/Configs/config_jvs002_{dataset_name}.yml`


In [1]:
import sys
import yaml
from pathlib import Path

# プロジェクトルートを取得
# プロジェクトルートを取得
# notebooks/styletts2/ から実行される場合: 2階層上がる
# notebooks/ から実行される場合: 1階層上がる
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent  # notebooks/styletts2 -> notebooks -> project_root
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent  # notebooks -> project_root
else:
    PROJECT_ROOT = current_dir  # プロジェクトルートから直接実行
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

# データセット名のリスト
DATASETS = ["phone_min4", "feat_top10", "full100"]

print("="*60)
print("設定ファイル生成開始")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"データセット: {DATASETS}")


設定ファイル生成開始

プロジェクトルート: /mnt/c/dev/minimal-feature-corpus-tts
StyleTTS2ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2
データセット: ['phone_min4', 'feat_top10', 'full100']


In [2]:
# ベース設定の定義（RTX 4070ti最適化）
BASE_CONFIG = {
    'log_dir': 'Models/jvs002_{dataset_name}',  # 後で置換
    'first_stage_path': 'first_stage.pth',
    'save_freq': 10,
    'log_interval': 10,
    'device': 'cuda',
    'epochs_1st': 80,  # Stage 1エポック数
    'epochs_2nd': 40,  # Stage 2エポック数
    'batch_size': 6,  # VRAM 12GBに最適化
    'max_len': 300,  # メモリ使用量削減
    'pretrained_model': '',  # ゼロから学習のため空文字
    'second_stage_load_pretrained': False,  # Stage 1から開始
    'load_only_params': False,
    
    'F0_path': 'Utils/JDC/bst.t7',
    'ASR_config': 'Utils/ASR/config.yml',
    'ASR_path': 'Utils/ASR/epoch_00080.pth',
    'PLBERT_dir': 'Utils/PLBERT/',
    
    'data_params': {
        'train_data': 'Data/jvs002_{dataset_name}/train_list.txt',  # 後で置換
        'val_data': 'Data/jvs002_{dataset_name}/val_list.txt',  # 後で置換
        'root_path': 'Data/jvs002_{dataset_name}/wavs',  # 後で置換
        'OOD_data': 'Data/OOD_texts.txt',
        'min_length': 50,
    },
    
    'preprocess_params': {
        'sr': 24000,
        'spect_params': {
            'n_fft': 2048,
            'win_length': 1200,
            'hop_length': 300,
        },
    },
    
    'model_params': {
        'multispeaker': False,
        'dim_in': 64,
        'hidden_dim': 512,
        'max_conv_dim': 512,
        'n_layer': 3,
        'n_mels': 80,
        'n_token': 178,  # 音素トークン数
        'max_dur': 50,
        'style_dim': 128,
        'dropout': 0.2,
        
        'decoder': {
            'type': 'istftnet',
            'resblock_kernel_sizes': [3, 7, 11],
            'upsample_rates': [10, 6],
            'upsample_initial_channel': 512,
            'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
            'upsample_kernel_sizes': [20, 12],
            'gen_istft_n_fft': 20,
            'gen_istft_hop_size': 5,
        },
        
        'slm': {
            'model': 'microsoft/wavlm-base-plus',
            'sr': 16000,
            'hidden': 768,
            'nlayers': 13,
            'initial_channel': 64,
        },
        
        'diffusion': {
            'embedding_mask_proba': 0.1,
            'transformer': {
                'num_layers': 3,
                'num_heads': 8,
                'head_features': 64,
                'multiplier': 2,
            },
            'dist': {
                'sigma_data': 0.2,
                'estimate_sigma_data': True,
                'mean': -3.0,
                'std': 1.0,
            },
        },
    },
    
    'loss_params': {
        'lambda_mel': 5.0,
        'lambda_gen': 1.0,
        'lambda_slm': 1.0,
        'lambda_mono': 1.0,
        'lambda_s2s': 1.0,
        'TMA_epoch': 50,
        'lambda_F0': 1.0,
        'lambda_norm': 1.0,
        'lambda_dur': 1.0,
        'lambda_ce': 20.0,
        'lambda_sty': 1.0,
        'lambda_diff': 1.0,
        'diff_epoch': 20,
        'joint_epoch': 50,
    },
    
    'optimizer_params': {
        'lr': 0.0001,
        'bert_lr': 0.00001,
        'ft_lr': 0.00001,
    },
    
    'slmadv_params': {
        'min_len': 400,
        'max_len': 500,
        'batch_percentage': 0.5,
        'iter': 10,
        'thresh': 5,
        'scale': 0.01,
        'sig': 1.5,
    },
    
    # RTX 4070ti最適化: Mixed precision training
    'fp16_run': True,
}

print("✓ ベース設定を定義しました")
print(f"\n主要パラメータ:")
print(f"  batch_size: {BASE_CONFIG['batch_size']}")
print(f"  max_len: {BASE_CONFIG['max_len']}")
print(f"  epochs_1st: {BASE_CONFIG['epochs_1st']}")
print(f"  epochs_2nd: {BASE_CONFIG['epochs_2nd']}")
print(f"  fp16_run: {BASE_CONFIG['fp16_run']}")


✓ ベース設定を定義しました

主要パラメータ:
  batch_size: 6
  max_len: 300
  epochs_1st: 80
  epochs_2nd: 40
  fp16_run: True


In [3]:
# 設定ファイル生成関数
def generate_config(dataset_name: str):
    """
    データセット名に対応する設定ファイルを生成
    
    Args:
        dataset_name: データセット名
    """
    print(f"\n{'='*60}")
    print(f"設定ファイル生成: {dataset_name}")
    print(f"{'='*60}")
    
    # 設定のコピーを作成
    config = BASE_CONFIG.copy()
    
    # 深いコピーが必要な部分
    import copy
    config['data_params'] = copy.deepcopy(BASE_CONFIG['data_params'])
    config['preprocess_params'] = copy.deepcopy(BASE_CONFIG['preprocess_params'])
    config['model_params'] = copy.deepcopy(BASE_CONFIG['model_params'])
    config['loss_params'] = copy.deepcopy(BASE_CONFIG['loss_params'])
    config['optimizer_params'] = copy.deepcopy(BASE_CONFIG['optimizer_params'])
    config['slmadv_params'] = copy.deepcopy(BASE_CONFIG['slmadv_params'])
    
    # データセット名を置換
    config['log_dir'] = config['log_dir'].format(dataset_name=dataset_name)
    config['data_params']['train_data'] = config['data_params']['train_data'].format(dataset_name=dataset_name)
    config['data_params']['val_data'] = config['data_params']['val_data'].format(dataset_name=dataset_name)
    config['data_params']['root_path'] = config['data_params']['root_path'].format(dataset_name=dataset_name)
    
    # 設定ファイルのパス
    config_dir = STYLETTS2_DIR / "Configs"
    config_dir.mkdir(exist_ok=True)
    config_path = config_dir / f"config_jvs002_{dataset_name}.yml"
    
    # YAMLファイルとして保存
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    print(f"✓ 設定ファイルを生成しました: {config_path}")
    print(f"\n主要設定:")
    print(f"  log_dir: {config['log_dir']}")
    print(f"  train_data: {config['data_params']['train_data']}")
    print(f"  val_data: {config['data_params']['val_data']}")
    print(f"  root_path: {config['data_params']['root_path']}")
    print(f"  batch_size: {config['batch_size']}")
    print(f"  epochs_1st: {config['epochs_1st']}")
    print(f"  epochs_2nd: {config['epochs_2nd']}")
    
    return config_path

print("✓ 設定ファイル生成関数を定義しました")


✓ 設定ファイル生成関数を定義しました


In [4]:
# 各データセットの設定ファイルを生成
config_paths = {}

for dataset_name in DATASETS:
    try:
        config_path = generate_config(dataset_name)
        config_paths[dataset_name] = config_path
    except Exception as e:
        print(f"✗ エラー: {dataset_name} の設定ファイル生成に失敗: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("全設定ファイル生成完了")
print("="*60)
for name, path in config_paths.items():
    print(f"  {name}: {path}")



設定ファイル生成: phone_min4
✓ 設定ファイルを生成しました: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_phone_min4.yml

主要設定:
  log_dir: Models/jvs002_phone_min4
  train_data: Data/jvs002_phone_min4/train_list.txt
  val_data: Data/jvs002_phone_min4/val_list.txt
  root_path: Data/jvs002_phone_min4/wavs
  batch_size: 6
  epochs_1st: 80
  epochs_2nd: 40

設定ファイル生成: feat_top10
✓ 設定ファイルを生成しました: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_feat_top10.yml

主要設定:
  log_dir: Models/jvs002_feat_top10
  train_data: Data/jvs002_feat_top10/train_list.txt
  val_data: Data/jvs002_feat_top10/val_list.txt
  root_path: Data/jvs002_feat_top10/wavs
  batch_size: 6
  epochs_1st: 80
  epochs_2nd: 40

設定ファイル生成: full100
✓ 設定ファイルを生成しました: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_full100.yml

主要設定:
  log_dir: Models/jvs002_full100
  train_data: Data/jvs002_full100/train_list.txt
  val_data: Data/jvs002_full100/val_list.txt
  root_path: Data/jvs002_full100

In [5]:
# 生成された設定ファイルの確認
print("\n" + "-"*60)
print("生成された設定ファイルの確認")
print("-"*60)

for dataset_name in DATASETS:
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    
    if config_path.exists():
        print(f"\n{dataset_name}: {config_path}")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)
        
        print(f"  log_dir: {config.get('log_dir', 'N/A')}")
        print(f"  batch_size: {config.get('batch_size', 'N/A')}")
        print(f"  max_len: {config.get('max_len', 'N/A')}")
        print(f"  epochs_1st: {config.get('epochs_1st', 'N/A')}")
        print(f"  epochs_2nd: {config.get('epochs_2nd', 'N/A')}")
        print(f"  fp16_run: {config.get('fp16_run', 'N/A')}")
        print(f"  train_data: {config.get('data_params', {}).get('train_data', 'N/A')}")
        print(f"  val_data: {config.get('data_params', {}).get('val_data', 'N/A')}")
    else:
        print(f"\n{dataset_name}: ファイルが見つかりません")



------------------------------------------------------------
生成された設定ファイルの確認
------------------------------------------------------------

phone_min4: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_phone_min4.yml
  log_dir: Models/jvs002_phone_min4
  batch_size: 6
  max_len: 300
  epochs_1st: 80
  epochs_2nd: 40
  fp16_run: True
  train_data: Data/jvs002_phone_min4/train_list.txt
  val_data: Data/jvs002_phone_min4/val_list.txt

feat_top10: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_feat_top10.yml
  log_dir: Models/jvs002_feat_top10
  batch_size: 6
  max_len: 300
  epochs_1st: 80
  epochs_2nd: 40
  fp16_run: True
  train_data: Data/jvs002_feat_top10/train_list.txt
  val_data: Data/jvs002_feat_top10/val_list.txt

full100: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_full100.yml
  log_dir: Models/jvs002_full100
  batch_size: 6
  max_len: 300
  epochs_1st: 80
  epochs_2nd: 40
  fp16_run: True
  train_data: Data/jvs0

## 設定ファイル生成完了

各データセットの設定ファイルが `StyleTTS2/Configs/config_jvs002_{dataset_name}.yml` に生成されました。

### VRAMエラーが発生した場合

もし学習中にVRAMエラーが発生した場合は、以下のパラメータを削減してください:

- `batch_size`: 6 → 4
- `max_len`: 300 → 200

設定ファイルを直接編集するか、このノートブックで再生成してください。

次のステップ:
- `03_train_models.ipynb` で学習を実行
